In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('Data_for_UCI_named.csv')

In [34]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


## Cheeck for missing values

In [35]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [36]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [37]:
df_unstable = df[df.stabf== 'unstable' ]
df_stable = df[df.stabf== 'stable' ]
data_df = df_unstable.append(df_stable) 

C:\Users\dell\AppData\Local\Temp\ipykernel_15472\3744699489.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = df_unstable.append(df_stable)


In [38]:
import sklearn.utils
df = sklearn.utils.shuffle(data_df)
df = data_df.reset_index(drop= True )
df.shape
df.stabf.value_counts() 


unstable    6380
stable      3620
Name: stabf, dtype: int64

In [39]:
df.stabf = df.stabf.astype('category')
df.stabf = df.stabf.cat.codes
df.stabf.value_counts()

1    6380
0    3620
Name: stabf, dtype: int64

In [40]:
# More processing 
df = df.drop(columns=['stab'])
X = df.drop(columns='stabf')
y = df['stabf']
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1
1,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,1
2,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,1
3,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,1
4,6.710166,3.765204,6.929314,8.818562,2.397419,-0.614590,-1.208826,-0.574004,0.177890,0.397977,0.402046,0.376630,1


In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

print('X_train shape: {}'.format(x_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(x_test.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [42]:
#transform train and test set using standard scaler
#ignore scaling for y since sklarn preprocessing ignores the y variable and it is also an object type.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [43]:
#transform train and test set using standard scaler
#ignore scaling for y since sklarn preprocessing ignores the y variable and it is also an object type.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [44]:
#put the scaled sets into a data_frame
x_train_scaled = pd.DataFrame(x_train_scaled, columns = x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns = x_test.columns)

# XGBoost For Training

In [45]:
from xgboost import XGBClassifier
xbc= XGBClassifier(random_state = 1)

#fit on train set
xbc.fit(x_train_scaled, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=1,
              reg_alpha=0, reg_lambda=1, ...)

In [46]:
#predict on test set
xbc_pred = xbc.predict(x_test_scaled)

## Measuring Model Performace for XGB Classifier

In [47]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score,confusion_matrix, classification_report

#### Accuracy

In [48]:
xbc_accuracy = accuracy_score(y_test, xbc_pred)
print('Accuracy: {}'.format(round(xbc_accuracy*100), 2))

Accuracy: 94


#### Precision

In [49]:
xbc_precision = precision_score(y_test, xbc_pred, pos_label= 0)
print('Precision: {}'.format(round(xbc_precision*100), 2))  

Precision: 92


#### Recall

In [50]:
xbc_recall = recall_score(y_test, xbc_pred, pos_label= 0)
print('Recall: {}'.format(round(xbc_recall*100), 2))

Recall: 91


#### F1 Score

In [51]:
xbc_f1 = f1_score(y_test, xbc_pred, pos_label= 0)
print('F1: {}'.format(round(xbc_f1*100), 2))

F1: 91


#### Classification Report

In [52]:
print('Classification Report:\n', classification_report(y_test,xbc_pred, digits =4))

Classification Report:
               precision    recall  f1-score   support

           0     0.9194    0.9104    0.9148       714
           1     0.9505    0.9557    0.9531      1286

    accuracy                         0.9395      2000
   macro avg     0.9349    0.9330    0.9340      2000
weighted avg     0.9394    0.9395    0.9394      2000



#### Confusion Matrix

In [53]:
xbc_cnf_mat = confusion_matrix(y_test, xbc_pred)
print('Confusion Matrix:\n', xbc_cnf_mat)

Confusion Matrix:
 [[ 650   64]
 [  57 1229]]


# LightGbm

In [54]:
from lightgbm import LGBMClassifier

lgbm= LGBMClassifier(random_state = 1)

#fit on train set
lgbm.fit(x_train_scaled, y_train)

LGBMClassifier(random_state=1)

In [55]:
#predict on test set
lgbm_pred = lgbm.predict(x_test_scaled)

### Model Performance for LightGBM Classifier

#### Accuracy

In [56]:
lgbm_accuracy = accuracy_score(y_test, lgbm_pred)
print('Accuracy: {}'.format(round(lgbm_accuracy*100), 2))

Accuracy: 94


#### Precision

In [57]:
lgbm_precision = precision_score(y_test, lgbm_pred, pos_label= 0)
print('Precision: {}'.format(round(lgbm_precision*100), 2))  

Precision: 93


#### Recall

In [58]:
lgbm_recall = recall_score(y_test, lgbm_pred, pos_label= 0)
print('Recall: {}'.format(round(lgbm_recall*100), 2))

Recall: 91


#### F1 Score

In [59]:
lgbm_f1 = f1_score(y_test, lgbm_pred, pos_label= 0)
print('F1: {}'.format(round(lgbm_f1*100), 2))

F1: 92


#### Classification Report

In [60]:
print('Classification Report:\n', classification_report(y_test,lgbm_pred, digits =4))

Classification Report:
               precision    recall  f1-score   support

           0     0.9269    0.9062    0.9164       714
           1     0.9485    0.9603    0.9544      1286

    accuracy                         0.9410      2000
   macro avg     0.9377    0.9333    0.9354      2000
weighted avg     0.9408    0.9410    0.9408      2000



#### Confusion Matrix

In [61]:
lgbm_cnf_mat = confusion_matrix(y_test, lgbm_pred)
print('Confusion Matrix:\n', lgbm_cnf_mat)

Confusion Matrix:
 [[ 647   67]
 [  51 1235]]


Extra tree Classifier


In [62]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(random_state = 1)

#fit on the train set
etc.fit(x_train_scaled, y_train)

ExtraTreesClassifier(random_state=1)

In [63]:
#predict on test set
etc_pred = etc.predict(x_test_scaled)

### Measuring Model Performance for ExtraTreeClassifier

In [65]:
#model accuracy
etc_accuracy = accuracy_score(y_test, etc_pred)
print('Accuracy: {}'.format(round(etc_accuracy*100), 2))

#precision
etc_precision = precision_score(y_test, etc_pred, pos_label= 0)
print('Precision: {}'.format(round(etc_precision*100), 2))  

#recall
etc_recall = recall_score(y_test, etc_pred, pos_label= 0)
print('Recall: {}'.format(round(etc_recall*100), 2))

#F1 score
etc_f1 = f1_score(y_test, etc_pred, pos_label= 0)
print('F1: {}'.format(round(etc_f1*100), 2))

#classification report
print('Classification Report:\n', classification_report(y_test,etc_pred, digits =4))

#confusion matrix
etc_cnf_mat = confusion_matrix(y_test, etc_pred)
print('Confusion Matrix:\n', etc_cnf_mat)

Accuracy: 93
Precision: 95
Recall: 83
F1: 89
Classification Report:
               precision    recall  f1-score   support

           0     0.9506    0.8347    0.8889       714
           1     0.9141    0.9759    0.9440      1286

    accuracy                         0.9255      2000
   macro avg     0.9323    0.9053    0.9164      2000
weighted avg     0.9271    0.9255    0.9243      2000

Confusion Matrix:
 [[ 596  118]
 [  31 1255]]


In [66]:
print("Training set score: {:.3f}".format(etc.score(x_train_scaled, y_train)))
print("Test set score: {:.3f}".format(etc.score(x_test_scaled, y_test)))

Training set score: 1.000
Test set score: 0.925
